In [4]:
from utils.hints import hide_toggle

ModuleNotFoundError: No module named 'utils.hints'

# Набор данных "Auction Verification"

Датасет был создан в рамках научного исследования с целью выяснить, можно ли заменить дорогостоящую верификацию сложных моделей процессов (в данном случае: одновременные многораундовые аукционы, используемые для аукционов частотных спектров) предсказаниями результатов.

Каждый экземпляр в этом датасете представляет один запуск верификации. Верификация выполняется для проверки, является ли определенная цена возможной для определенного продукта, и в некоторых случаях, определения, может ли определенный участник выиграть продукт по этой цене. Каждый экземпляр содержит детали одного такого сценария верификации.

Краткое описание признаков:

- process.b1.capacity: Характеристика (вместимость) первого участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 1.

- process.b2.capacity: Характеристика (вместимость) второго участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 2.

- process.b3.capacity: Характеристика (вместимость) третьего участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 3.

- process.b4.capacity: Характеристика (вместимость) четвертого участника (заявителя). Целое число. Вместимость (максимальное количество продуктов для выигрыша) участника 4.

- property.price: Характеристика (цена) продукта. Целое число. Текущая проверенная цена.

- property.product: Характеристика (продукт) продукта. Целое число. Текущий проверенный продукт.

- property.winner: Характеристика (победитель) продукта. Целое число. Участник (заявитель), который в настоящее время признан победителем продукта (0, если проверена только цена).

Целевая переменная:

- verification.result: Целевой атрибут (категориальный). Результат верификации. Бинарный результат - возможен ли проверенный результат?

In [1]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных auction из CSV-файла в DataFrame
df = pd.read_csv("data/auction.csv")
df.drop(columns=["verification.time"], inplace=True)


# # Определение признаков (X) и целевой переменной (y)
X = df.drop(columns=["verification.result"])
y = df["verification.result"].astype(int)

# Нормализация признаков с помощью StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Создание модели KNN для регрессии
knn = KNeighborsClassifier()

# Определение сетки параметров для перебора (n_neighbors и weights)
param_grid = {"n_neighbors": np.arange(1, 10), "weights": ["uniform", "distance"]}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(X_test)

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test)

# Вывод значения f1
print("f1 на тестовом наборе: ", f1)

Наилучшие параметры на кросс-валидации: {'n_neighbors': 3, 'weights': 'uniform'}
Наилучшая оценка на кросс-валидации: 0.8825012663927506
f1 на тестовом наборе:  0.3116883116883117


In [2]:
df.head()

,process.b1.capacity,process.b2.capacity,process.b3.capacity,process.b4.capacity,property.price,property.product,property.winner,verification.result
0,0,0,2,1,59,1,0,False
1,0,0,2,1,59,2,0,False
2,0,0,2,1,59,4,0,False
3,0,0,2,1,59,6,0,False
4,0,0,2,1,60,1,0,True


# Набор данных "Predict students' dropout and academic success"

Датасет был создан в рамках проекта, направленного на снижение количества студентов, прекращающих обучение или испытывающих неудачи в высшем образовании, путем использования методов машинного обучения для выявления студентов, находящихся в рискованных ситуациях на раннем этапе их учебного пути, чтобы разработать стратегии для их поддержки.

Датасет включает информацию, известную на момент поступления студента в учебное заведение, такую как учебный путь, демографические данные и социоэкономические факторы.

Задача сформулирована как задача классификации с тремя категориями (бросившие обучение, зачисленные и выпускники) к концу нормативного срока обучения. То есть, для каждого студента предсказывается, к какой из трех категорий он относится к моменту окончания курса (нормативного срока обучения).

Краткое описание признаков можно посмотреть по ссылке: https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success

Целевая переменная:

- **Target**: бросившие обучение, зачисленные и выпускники

In [5]:
"""
По аналогии с решением задачи Auction Verification решите в текущей ячейке задачу для предсказания целевой переменной на 
наборе Bike Sharing Demand. С случае затруднений можно посмотреть на образец решения задания в ячейке ниже.
"""

# Импорт необходимых библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных auction из CSV-файла в DataFrame
df = pd.read_csv("data/students_predictions.csv", sep=";")
df.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [6]:
# # Определение признаков (X) и целевой переменной (y)
X = df.drop(columns=["Target"])
y = df["Target"].astype("category")

# Нормализация признаков с помощью StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Создание модели KNN для регрессии
knn = KNeighborsClassifier()

In [7]:
# Определение сетки параметров для перебора (n_neighbors и weights)
param_grid = {"n_neighbors": np.arange(1, 10), "weights": ["uniform", "distance"]}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

Наилучшие параметры на кросс-валидации: {'n_neighbors': 9, 'weights': 'distance'}
Наилучшая оценка на кросс-валидации: 0.7061347781267231


In [9]:
# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(X_test)

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test, average="weighted")

# Вывод значения f1
print("f1 на тестовом наборе: ", f1)

f1 на тестовом наборе:  0.730611543320054


In [4]:
hide_toggle(for_next=True)

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных students predictions
numeric_features = []
categorical_features = []
df = pd.read_csv("data/students_predictions.csv", sep=";")

# Отделяем целевую переменную от признаков
X = df.drop(columns=["Target"])
y = df["Target"]

for column in X.columns:
    unique_values_count = df[column].nunique()
    if 2 < unique_values_count < 50:
        categorical_features.append(column)
    else:
        numeric_features.append(column)

numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Создание модели KNN для регрессии
knn = KNeighborsClassifier()

# Определение сетки параметров для перебора (n_neighbors и weights)
param_grid = {"n_neighbors": np.arange(1, 10), "weights": ["uniform", "distance"]}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(preprocessor.fit_transform(X_train), y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(preprocessor.transform(X_test))

# Вычисление среднеквадратической ошибки (MSE) на тестовом наборе
f1 = f1_score(y_pred, y_test, average="weighted")

# Вывод значения MSE
print("f1 на тестовом наборе: ", f1)

Наилучшие параметры на кросс-валидации: {'n_neighbors': 8, 'weights': 'uniform'}
Наилучшая оценка на кросс-валидации: 0.7256318973301689


/home/fitlemon/code/math_for_ds/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [5, 6, 7, 9, 10, 11, 16, 17, 18, 19] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


f1 на тестовом наборе:  0.7362983688318167


# Набор данных "accelerometer_gyro_mobile_phone_dataset"

Данные, собранные в 2022 году в Университете короля Сауда в Эр-Рияде для распознавания действий человека с помощью датчиков IMU мобильного телефона (акселерометр и гироскоп). эти действия сводятся к стоянию (остановке) или ходьбе.

Краткое описание признаков:
- accX	Вещественный
- accY	Вещественный
- accZ	Вещественный
- gyroX	Вещественный
- gyroY	Вещественный
- gyroZ	Вещественный
- timestamp	Datetime

Целевая переменная:

- **Activity**: стояние или ходьба

In [12]:
"""
По аналогии с решением задачи Auction Verification решите в текущей ячейке задачу для предсказания целевой переменной на 
наборе accelerometer_gyro_mobile_phone_dataset. С случае затруднений можно посмотреть на образец решения задания в ячейке ниже.
"""

# Импорт необходимых библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# Загрузка набора данных auction из CSV-файла в DataFrame
df = pd.read_csv("data/accelerometer_gyro_mobile_phone_dataset.csv", sep=",")
df.head()

,accX,accY,accZ,gyroX,gyroY,gyroZ,timestamp,Activity
0,-0.496517,3.785628,8.954828,-0.142849,-0.126159,-0.022539,34:22.9,1
1,-0.462388,3.869603,9.281898,0.084349,0.096695,0.092130,34:23.0,1
2,-0.296084,3.820505,8.930728,0.061763,0.051543,0.071287,34:23.1,1
3,-0.469723,3.890110,8.744067,0.007641,0.028679,0.109433,34:23.2,1
4,-0.472418,4.109105,8.941207,-0.123640,0.099057,0.051943,34:23.3,1


In [14]:
# # Определение признаков (X) и целевой переменной (y)
X = df.drop(columns=["Activity", "timestamp"])
y = df["Activity"].astype(int)

# Нормализация признаков с помощью StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Создание модели KNN для регрессии
knn = KNeighborsClassifier()

In [18]:
# Определение сетки параметров для перебора (n_neighbors и weights)
param_grid = {"n_neighbors": np.arange(1, 10), "weights": ["uniform", "distance"]}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.feature_names_in_)
print(grid_search)
# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

['accX' 'accY' 'accZ' 'gyroX' 'gyroY' 'gyroZ']
Наилучшие параметры на кросс-валидации: {'n_neighbors': 8, 'weights': 'distance'}
Наилучшая оценка на кросс-валидации: 0.982650854180088


In [16]:
# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(X_test)

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test, average="weighted")

# Вывод значения f1
print("f1 на тестовом наборе: ", f1)

f1 на тестовом наборе:  0.9900645488443383


In [7]:
hide_toggle(for_next=True)

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error

# Загрузка набора данных
df = pd.read_csv("data/accelerometer_gyro_mobile_phone_dataset.csv")

# Удаление исходного столбца "date"
df = df.drop(columns=["timestamp"])

# Отделяем целевую переменную от признаков
X = df.drop(columns=["Activity"])
y = df["Activity"]


# Предобработка данных: преобразование категориальных признаков и нормализация числовых признаков
numeric_features = X.columns.tolist()

numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_features)]
)


# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Создание модели KNN для регрессии
knn = KNeighborsClassifier()

# Определение сетки параметров для перебора (n_neighbors и weights)
param_grid = {"n_neighbors": np.arange(1, 10), "weights": ["uniform", "distance"]}

# Использование GridSearchCV для поиска наилучших параметров модели
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring="neg_mean_squared_error")
grid_search.fit(preprocessor.fit_transform(X_train), y_train)

# Вывод наилучших параметров и оценки на кросс-валидации
print("Наилучшие параметры на кросс-валидации:", grid_search.best_params_)
print("Наилучшая оценка на кросс-валидации:", grid_search.best_score_)

# Прогнозирование на тестовом наборе с помощью модели с оптимальными параметрами
y_pred = grid_search.predict(preprocessor.transform(X_test))

# Вычисление f1 на тестовом наборе
f1 = f1_score(y_pred, y_test)

# Вывод значения MSE
print("f1 на тестовом наборе: ", f1)

Наилучшие параметры на кросс-валидации: {'n_neighbors': 9, 'weights': 'uniform'}
Наилучшая оценка на кросс-валидации: -0.018247850436668638
f1 на тестовом наборе:  0.992266414141414
